In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainer

csv_file_path = '/kaggle/input/dataset/Dataset - Sinhala.csv'
df = pd.read_csv(csv_file_path)

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/sinmt5/transformers/default/1/sinMT5-tuned")

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True,padding=True, )

    labels = tokenizer(examples["summary"], max_length=150, truncation=True, padding=True, )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    seed=42,
    optim="adamw_torch",
    weight_decay=0.0,
    num_train_epochs=7,
    lr_scheduler_type="linear",
    logging_dir="./logs",
    save_total_limit=7,
    predict_with_generate=True,
    report_to="none",
    load_best_model_at_end=True
)


model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/sinmt5/transformers/default/1/sinMT5-tuned")


class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    def save_model(self, output_dir=None, _internal_call=False):
        # Ensure all tensors are contiguous before saving
        for name, param in self.model.named_parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()

        # Call the parent class's save_model() method
        super().save_model(output_dir, _internal_call=_internal_call)

trainer = CustomSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
)

# Start training (fine-tuning)
trainer.train()

In [ ]:
from transformers import pipeline

pipe = pipeline("summarization", model="/kaggle/working/results/checkpoint-966")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/results/checkpoint-966")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/results/checkpoint-966")

In [ ]:
def summarize_text(text):
    # Tokenize input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary using beam search (or any other decoding strategy)
    summary_ids = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)

    # Decode summary back into text format
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

In [ ]:
sinhala_text = """
ඉදිරි කාලසීමාවේදී වාහන ආනයනය කිරීම් ආරම්භ කිරීමත් සමඟ වාහන ආනයන බදු ද
ඉහල දැමීමට රජය තීරණය කර තිබේ. මෙරට ජීවන වියදම සලකා බලමින් වාහන ආනයන බදු සංශෝධනය
කිරීමටත් ජාත්‍යන්තර මුල්‍ය අරමුදල සමග මේ වන විටත් රජය එකඟ වී සිටී. මෙම බද්ද සංශෝධනය කරනු
ලබන්නේ ආර්ථික වර්ධනය හා උද්ධමන අනුපාතයන් ගලපමිනි. කෙසේ වෙතත් අත්‍යවශ්‍ය වාහන පමණක් මුලින්
ආනයනය කිරීමට රජයේ අවධානය යොමු වී ඇති අතර පුද්ගලික වාහන ආනයනය කරන්නේ නම් එන්ජින් ධාරිතාව
1000ට අඩු කුඩා වාහන පළමුව ආනයනය කිරීමට සැලසුම් කර තිබේ.
"""

summary = summarize_text(sinhala_text)
print("Summary:", summary)

In [ ]:
text2 = """
තෙල් වගේ හාල් හිගයක් ඇති කරොත්, ඒකත් එක්ක fight කරන කතාව ගත්තොත් අපි ලග මොකුත් නැතුව අපිට fight කරන්න බෑ.
මොකද නීතියෙන් විතරක් මේව කරන්න බෑ. එයාල ලග වී තොග තියනවා, වී මෝල් තියනවා.
ඉතින් අපි ඒ ගොල්ලන් එක්ක fight කරන්න නම් අපිත් මේ market එකේ කොටසක් අත්පත් කර ගන්න ඕන.
market එකේ කොටසක් අත්පත් කර ගත්තාම මේ ගොල්ලොන්ට මේ, මම දන්නවා සමහරවිට දැන් රුපියලක් අඩුවෙන් විකුණනවට
වඩා ඒ අයට පාඩුයි මෝල වහල තියන එක. හරිද, සතියක් ඉදියි. මොකද ඒගොල්ලො මේ බැංකු වලින් ලෝන් අරන් තියෙන්නේ.
ඒ ගොල්ලො market එකට එන්න ඕන. ඒ ගොල්ලො එක්ක වැඩ කරනවා මිනිස්සු. වැඩ කරන මිනිස්සුන්ට පඩි ගෙවන්න ඕන.
එතකොට සමහරු ස්ථාවර විදුලිය ලබා ගැනීම සදහා විවිධ transformers එහෙම හයි කරන් තියනවා. ඒවට වැඩිපුර වියදමක් දරන්න වෙලා තියනවා.
ඒ නිසා මෝල්  ක්‍රියාත්මක කරන්න ඕන. මෝල් ක්‍රියාත්මක කරන්නේ නැතුව වහගෙන ඉන්න බෑ මෝල් හිමියන්ට.
හැබැයි ඒ ගොල්ලොන්ගෙ ක්‍රියාත්මක කිරීම නියාමනය කරන්න පුළුවන්කමක් තියනවා කොතනින්ද, ආණ්ඩුව market එක අත්පත් කර ගැනීමෙන්. ඒක සැලසුමක්.
"""

summary = summarize_text(text2)
print("Summary:", summary)

In [ ]:
text3 = """
අංක 1 මම අධ්‍යාපනය පැත්තෙන් මගේ අවශ්‍ය සුදුසුකම් ටික සම්පූර්ණ කරලා තියෙනවා. ඒ වගේම දිස්ත්‍රික්කය තුළ වසර දහයක පමණ කාල පරිච්ඡේදයක් මාතර මහ පොළොවේ හැම ගම්මානයකට ම ගිහිල්ලා මහජන අවශ්‍යතා පිළිබඳ හොඳ අදහසක් සහ දේශපාලන ආභාෂයක් තියෙන නිසා මම විශ්වාස කරනවා මේ රටේ මිනිස්සුන්ට අවශ්‍ය ප්‍රතිසංස්කරණ රජයට ගේන්න, රජය පාර්ලිමේන්තුවක් තුළ මහජනතාව වෙනුවෙන් හඬ නගන්න මට අවශ්‍ය සුදුසුකම් ටික තියෙනවා. එකෙන් මම දකින්න නෑ මම සුදුසුයි කියන කාරණය. මොකද හේතුව ලක්ෂ්මන් යාපා ගේ පුතා වීම ඇත්තටම දේශපාලන ජීවිතය ආරම්භ කිරීමට අවශ්‍යතාවයක් වුණා. ඒක මට ලැබිච්ච දෙයක් සාමාන්‍යයෙන් ගුරුවරයෙකු පුතෙක් ආසයි ගුරුවරයෙක් වෙන්න, විද්‍යාඥයෙක්ගේ පුතෙක් ආසයි එයාර්පෝට් එකට යන්න ඒ වගේ තියෙනවා අපි පොඩි කාලේ දේශපානයට ආසයි. තාත්තාගේ පුතා වෙලා ඉගෙන ගන්නෙත් නැතුව මහපොළොව ඇවිදින්නෙත් නැතුව තාත්තගේ ජනප්‍රියත්වය විතරක් මම යනවා නම් ඒක සුදුසු නෑ. නමුත් මමත් අවුරුදු දහයක් මාතර දිස්ත්‍රික්කයේ හැම ගමකටම ගියපු තරුණයෙක්. පසද කිව්වම ලංකාවත් දන්නවා ඇති මාතර දිස්ත්‍රික්කය හොඳටම දන්නවා.
"""

summary = summarize_text(text3)
print("Summary:", summary)

In [ ]:
text4 = """
ශ්‍රීලන්කන් ගුවන් යානා කිහිපයක් අවලංගු කිරීම සම්බන්ධයෙන් කිව හැක්කේ කුමක්ද?
කටුනායක ගුවන්තොටුපොළ වෙත පැමිණි ශ්‍රීලන්කන් ගුවන් යානා කිහිපයක ඇති වු තාක්ෂණික දෝෂ හේතුවෙන් ගුවන් ගමන් රැසක් අවලංගු කර තිබේ.‍
ගුවන් යානා 3ක් තාක්ෂණික දෝෂවලට ලක්වීම නිසා, ගුවන් ගමන් වාර කිහිපයක් අවලංගු කර ඇති අතර තවත් ගුවන් ගමන් වාර කිහිපයක් ප්‍රමාදවීම්වලට ලක් වී ඇති බව ශ්‍රීලන්කන් ගුවන් සේවා ප්‍රකාශකවරයෙකු පැවසුවේය.
සෑම ගුවන් යානයක්ම තම ගුවන් ගමන නිමා කිරීමෙන් පසු තාක්ෂණික ඇගයීම් පරීක්ෂා කිරීමකට ලක්කෙරෙන අතර, එහිදී ඇතැම් ගුවන් යානාවල සිදුකළ යුතු අත්‍යාවශ්‍ය තාක්ෂණික නඩත්තු කිරීම් සඳහා තරමක කාලයක් ගත වන බවත්, ඒ අනුව මෙම ගුවන් යානා 3 එම තාක්ෂණික නඩත්තු කටයුතු සඳහා යොමු කර ඇති බව ද එම ප්‍රකාශකවරයා වැඩිදුරටත් සඳහන් කළේය.
මේ හේතුවෙන් ඊයේ (17) රාත්‍රී 06.35ට ඉන්දියාවේ චෙන්නායි බලා පිටත්ව යාමට නියමිත ශ්‍රීලන්කන් ගුවන් සේවයේ යූ.එල්- 123 දරන ගුවන් ගමන සහ යළිත් රාත්‍රී 10.15ට චෙන්නායි සිට කටුනායක වෙත පැමිණීමට නියමිත ශ්‍රීලන්කන් ගුවන් සේවයේ යූ.එල්- 124 ගුවන් ගමන අවලංගු කර ඇති බව සඳහන්ය.
මීට අමතරව රාත්‍රී 10.10ට ඉන්දියාවේ නවදිල්ලි නුවර සිට කටුනායක ගුවන්තොටුපොළ වෙත පැමිණීමට නියමිත ශ්‍රීලන්කන් ගුවන් සේවයේ යූ.එල් - 196 දරන ගුවන් ගමන සහ ඔස්ට්‍රේලියාවේ මෙල්බර්න් නුවර සිට රාත්‍රී 10.20ට කටුනායක ගුවන්තොටුපොළ වෙත පැමිණීමට නියමිත ගුවන් ගමන ද අවලංගු කර තිබේ.
මේ පිළිබඳ ශ්‍රීලන්කන් ගුවන් සේවය, මෙම ගුවන් යානාවලින් ගමන් පහසුකම් සලසා ගන්නා මගීන් දැනුවත් කිරීමට පියවර ගෙන ඇති අතර, කඩිනමින් මෙම ගුවන් ගමන්වල නිරත වීමට අවශ්‍ය ගුවන් මගීන්, වෙනත් විකල්ප ගුවන් යානා මඟින්, එම ගමනාන්තයන් වෙත යොමු කිරීමට ද ශ්‍රීලන්කන් ගුවන් සේවය පියවර ගෙන තිබුණි.
"""

summary = summarize_text(text4)
print("Summary:", summary)

In [ ]:
text5 = """
මේ වෙනකොට ලෝකෙ පුරාම ගොඩක් coding tools ජනප්‍රිය වෙලා තියෙනවා. ගොඩක් software engineers මේවා පාවිච්චි කර තමන්ගේ efficiency ක වැඩි කරගෙන තියෙනවා. මේ අතරෙ software engineering graduates la ඉන්නවා company internship එකක් ලබා ගන්න බැරුව මේක ටිකක් සංකීර්ණ තත්වයන් අපි බලමු මේකට මොකක්ද කරන්නේ කියලා. මේ ප්‍රශ්නය හරියටම තේරුම් ගන්න අපි මේක දිහා පැති කීපයකින් බලන්න ඕන අපි මුලින්ම බලන්න ඕන natural language coding, ඒ කියන්නෙ සාමාන්‍යයෙන් කතා කරන භාෂාවෙන් code ලියන්නෙ කොහොමද? ඒකෙ පරිණාමයේ මොකක්ද කියලා. ඊළඟට අපි තේරුම් ගන්න software development process එකේ තියන steps මොනාද කියල. මේ steps එක්ක බැඳිච්ච job roles තියනවා, ඒ ගොල්ලොත් එක්ක බැදිච්ච වගකීම් තියෙනවා, මේ හැම දෙයක්ම තේරුම් ගන්න උත්සාහ කරන්න ඕන. අන්තිමට අපි තේරුම් ගන්න ඕන දැන් ඉන්න software engineering graduates කෙනෙක්ගෙ දැනුම සහ අලුත් industry එකේ standard අතර මුහුණ මගේ පරතරයක තියෙන්නෙ කියලා මේ කාරණාවක්ම තේරුම් ගත්තොත් එහෙම අපිට මේ ප්‍රශ්නෙදි සමග ගන්න මාර්ගයක් පාදා ගන්න පුළුවන් වෙයි. මීට දශක කීපයකට කලින් අපි computer programming වලට පාවිච්චි කලේ මැෂින් language එක. මේක හසුරවන්න දන්න professionals ල ඉතාමත් සීමිත ප්‍රමාණයක් විතරයි හිටියේ. ඒ ගොල්ලන්ටත් විදිහට ලියන්න ඉතාමත් දිග කාලයක් වැය වුණා. කාලයත් එක්ක මේ programming languages ක්‍රම ක්‍රමයෙන් පරිණාමය වෙන්න පටන් ගත්තා අපි assembly programming දකින්න හම්බුනා ඊට පස්සේ imparative programming දකින්න හම්බුනා. ඊටත් පස්සේ object oriented programming දකින්න හම්බුනා මේ විදිහට programming languages පරම්පරාවලින් දියුණු වෙනකොට එක ක්‍රමක්‍රමයෙන් natural language එකට කිට්ටු වුණා. ඒ කියන්නෙ අපි සාමාන්‍ය කතා කරන ඉංග්‍රීසි භාෂාවට a programming language එක ගොඩක් කිට්ටු වුණා. ඒ හින්දම ලෝකය පුරා ඉන්න ගොඩක් අයට computer programming කරන්න පුළුවන් වුණා. දශක ගණනක් පුරාවටම පරිගණක විද්‍යාඥයින් මේ computer programming language එක natural language එකට, ඒ කියන්නෙ අපි සමාන්‍යයෙන් පාවිච්චි කළේ ඉංග්‍රීසි භාෂාවට පුළුවන් තරම් සමීප කරන්න මහන්සි වුනා. ඒක ප්‍රතිඵලයක් විදියට තමයි අද අපි පාවිච්චි කරන Java, Python වගෙ programming languages හැදිලා තියෙන්නේ. හැබෙයි generative AI tools ජනප්‍රිය වෙනවත් එක්කම මේ තත්වය සම්පූර්ණයෙන්ම වෙනස් වුණා. පරිගණක භාෂාව ඒ කියන්නෙ programming languages එක සරල කරනවා වෙනුවට අපිට පුළුවන් වුණා AI tool එහෙකට සාමාන්‍ය භාෂාවෙන් විස්තර කළ කියන්න අපිට මොනවද කෝඩ් එකක් ද කියලා. එතකොට එ AI tool එකට පුළුවන් අපිට ඕන code eka generate කරලා දෙන්න. මේක IT professionals ලගෙ දශක ගණනක් පුරාවට දිග ගොඩක් ලොකු සිහිනයක් අද වෙනකොට ඒ සිහිනෙන් බොහෝ දුරට සැබෑ කරගන්න පුලුවන් වෙලා තියෙනවා. මූ කරනවා වෙන්න පුළුවන් තරම් ඉක්මනට ප්‍රශ්න එන්න කියලා අපි පිටුවෙන් පිටු ගුණක් software engineers ලා ඉන්න උන් code genarate දැන් විවිධා කාරයි how to use පාවිච්චි කරනවා ඔබල නතර ගමන් කරන මේ hai thumse ගැන හොයන්න කීම ගැන ඉගෙනගන්න කිව පාවිච්චි කළ කූජන රෙස්ට් කරන්න ඉගෙන ගන්න කියලා හැබැයි කොහොම වුණත් මේ වෙනකොට ගොඩක් software engineering graduates ලා ඉන්නවා ඩ්ග්‍රේ එක ඉවර කළ internship එකක් ලබාගන්න බලාපොරොත්තුවෙන්. ගොඩක් කට්ටිය මැසේජ් කරලා තියෙනවා මේගොල්ලො මේ වෙනුවෙන් මාස ගානක් බලාගෙන ඉන්නවා කියලා. ඉතින් මෙතන මොකද ඇත්තටම සිද්ධ වෙන්නේ මේ ප්‍රශ්නයට දින විසඳුම් මොකක්ද? අපි මේ කතාව තේරුම් ගන්න software development process එකේ තියන steps මොනවාද, ඒ වගෙ තියන ජොබ් රෝල් මොනවද? ගැන ගැබුරෙන් සාකච්ඡා කරන්න ඕන. මේ ගැන පුළුල් අවබෝධයක් ලබා ගත්තොත් එහෙම මේකට උත්තරේ මොකක්ද  කියාලා ඕගොල්ලොන්ටම හිතාගන්න පුලුවන් වෙයි. software develop කරන්නෙ මොකක් හරි ප්‍රශ්නයක් විසඳන්න. මේකෙ පලවෙනි පියවර තමයි මේ ප්‍රශ්නෙ මොකක්ද කියලා අඳුනගන්න එක. අපිට ගොඩක් වෙලාවට මේ වගේ ප්‍රශ්නයක් client කෙනෙක්ගෙ රික්වෙස්ට් එකක් විදිය එනවා. සමහර වෙලාවට මේ වගේ ප්‍රශ්නයක් අපිට RFP එකක් විදියට ඒ කියන්නෙ request for proposal එකක් විදියට එනවා. එහෙම නෙත්තම් ඔයාගෙන් brand new idea එකක් වෙන්නත් පුළුවන්. ඊළඟට ඇති කරන්නේ මේ ප්‍රොජෙක්ට් එක සම්බන්ධයෙන් feasibility study එකක් කන්න එක. ඒ කියන්නෙ ශක්‍යතා අද්‍ය්නාය වාර්තාවක්. මෙතනදී අපි technology, economy, leagality, operatorbility, scalebility වගේ factors ගොඩක් දිහා බලනවා. මේ හැම එකක්ම pass උනොත් එහෙම අපිට පුළුවන් ඊලග පියවරට යන්න. ඊළඟ පියවර තමයි requirement gathering and analysis. ගොඩක් ජොබ් කරන්න business analyst කෙනෙක්, මේ process eke output එක වෙන්නේ SRS එකක්, ඒ කියන්නෙ system requirement specification එකක්. මේ යනවා project manager ඒ වගෙම senior architect ට.  senior architect යටතේ වැඩ කරනවා architects ලා ඒ වගෙම associate architects ල. ඒගොල්ලොන්ගෙ ජොබ් එක වෙලා තියෙන්නේ මේ software එකට අවශ්‍ය ඩිසයින් සාදන එක, ඒ කියන්නෙ blueprints සාදන එක. software design software සියලුම විස්තර අඩංගු වෙනවා. ඉස්සර කාලේ අපි පාවිච්චි කර සමහර toools me design කිරීම කෝඩ් බවට පරිවර්තනය කරන්න පුළුවන්, හැබැයි ඒ tools වලට පුළුවන් වුන මේ ඩිසයින් එක දිහා බලාගෙන සම්පූර්ණ වූ එකම ලියන්න ඒ කොඩ් එකේ සැකිල්ල විතරයි හඩන්න පුලුවන් උනේ. මේ විදියට සොෆ්ට්වෙයාර්  එකේ සැකිල්ල හදා ගත්තට පස්සේ tech leads ලා,  senior software engineer ලා software engineers associate software engineers la development පටන් ගන්නවා. ඉතින් මේ විඩියට කොඩ් කරලා ඉවර වුනාට පස්සේ මේක ටෙස්ට් කරන්න ඕන, මේකට කිව්වා ප්ලෑන් එකක් හදන්න ඕනේ මේකෙ deployment එක සම්පූර්ණ කරන්න ඕන, software එක පාවිච්චි කරන යූසර්ස්ලා train කරන්න ඕනේ මේ මගින් පියවර විශාල ගණනක් තියෙනවා. ඉතින් මම මේ software development process ඒ ගැන ඉතාමත්ම අනුව පැහැදිලි කිරීමක් කලේ. 
"""

print( summarize_text(text5))

In [ ]:
text6 = """
අද මම කතා කරන්න ලැස්ති වෙලා ඉන්නෙ මේ ලෝකේ දැනට තියෙන හොඳම අධ්‍යාපන ක්‍රමවේද පිළිබඳව මොකද ඇය ඉගෙන ගත්තොත් අපි ගොඩක් දෙනෙක්ට ඒකෙන් ලොකු ප්‍රයෝජන ගන්න පුළුවන්කම තියෙන නිසා. මම දැන් ඕගොල්ලන්ට පෙන්නන්න යන්නේ මේ ලෝකයේ අධ්‍යාපන ක්‍රමයම ගිය සියවස තුළ ද වෙනස් කරපු පින්තූරක්. අපේ සාම්ප්‍රදායික අධ්‍යාපන ක්‍රම ප්‍රධාන පාසැල් වල උගන්වන අධ්‍යාපන ක්‍රමය ඇතුලේ පන්තියක ළමයින්ගෙ යම්කිසි විෂයකට ලකුණු පැතිරිලා තියෙන්නේ විදිහට කියල හිතන්න හිතන්න ළමයි ලකුණු 15 ක් ළමයිනුත් දෙන ලකුණු 95 ගත්තු ළමයිනුත් ඉන්නවා මේ වගේ ලොකු පැතිරුණු වක්‍රයක, ඉතින් වැඩිපුර ළමයින්ට වැඩිපුර සාමාන්‍ය ලකුණු 50 50 වගේ ගත්තු අය තමයි ඉන්නේ එකක ඉන්න B එකක ළමයි තමයි වැඩිපුරම ඉන්නෙ අපි සාමාන්‍යයෙන් දකින්නේ. මම අද කියලා දෙන්න යන මේ අධ්‍යාපන ක්‍රමය තුළ මේ ඒව ක්‍රමේ ළමයි ටික අලුතෙන් ළමයි නෙමෙ මේ ළමයින්ට කිසිම වෙනසක් කරන්නැතුව මේ ක්‍රමවේදය වෙනස් කිරීමෙන් පමණක් අපිට හැකියාව තියෙනවා මේ ලමයින්ගේ කුසලතාවය මේ යම්කිසි විෂයකට තියන ප්‍රවීනතාවය එකේ මනින  ලද ලකුණු වශයෙන් ගත්තම මෙන්න මේ විදිහට දකුණු අත පැත්තට යවන්න.  ගැන එහෙම කිව්වම තේරෙන්නෙ නෑනෙ ඒ ගැන කවුරුද හිතන්න B මාක් එකක් තියෙන,  B great එකක් තියෙන ළමයෙක්, 55 වගේ ලකුණු පනස් පත්තිනි ළමයෙක්,   සාමාන්‍ය ළමයෙක්, මධ්‍ය ළමයා. මෙන්න මේ ළමයා ඒ ළමයාගේ යම්කිසි මේ වේ තියන හැකියාව වෙනසක් නැතුව මේ අධ්‍යාපන ක්‍රමය වෙනස් කිරීමෙන් පමණක් A+ එකකට කිට්ටු කරන්න කියන්න පුළුවන්. මේක බොරුවක් ද මේ නැවත නැවත කරන ලද පරීක්ෂණ වලින් sauru දීමේ ක්‍රමවේදය හොඳයි හොඳම දැනට අපි දෙන හොඳම ක්‍රමවේදය වවා පැහැදිලි වෙලා තියෙන ඇත්තට කවුද නූතන අධ්‍යාපන ක්‍රමයේ framework නිහඬ වේගයන් වෙනස් අර්ථ දක්වපු පින්තූර ඉන්නවා මෙන්න මේ පින්තූරෙ අධ්‍යාපනය වෙනස් කරන්න ඕන වෙනස් කරන්න ඕනි කියලා කතා කරන හිතන්න මේ ක්‍රමය ඔබේ දරුවාට මෝඩම කරන්න පුළුවන් කියලා ඔබට පුද්ගලිකව බොමු කරගත්තා ගැන හිතන්න ගුරුවරයෙක් නම් ඔබ ගුරුවරයෙක් විදියට ඔබේ ශිෂ්‍යන්ට යොමු කරන්න පුළුවන් කළේ දන්නවා ඒක අමතක කරන්න පුළුවන් රටක් සමාජයක් සහෘදයින් සම්පූර්ණම වෙනස් කරන්න පුළුවන් ප්‍රබල ක්‍රමවේදයක් අද මම උඹට අර heroes අවුරුදු ඔය අධ්‍යාපන ප්‍රතිසංස්කරණ ගැන කතා කරන කාලයක් වැනි ඇතැම් පුළුල් ළමයින් ගත්තොත් හැබැයි මම පාසල් වලට යන ලමයි මොකක්ද උඹලගෙ අධ්‍යාපනය වෙනස් වෙන්න ඕනේ කියලා දෙමව්පියන්ගෙන් අහනවා නම් මම ඒ වගේම ඔයා අධ්‍යාපන කාර්යාල වල වැඩ කරන එළඳ හමුදාවට භූමිකාව මොකක්ද වෙනස් වෙන්න ඕන මිය මට අධ්‍යාපනය හොඳයි අර රටේ අධ්‍යාපනය අපිට වඩා හොඳයි මේ අධ්‍යාපනය වෙනස් කරන්ඩ ඕනෑ අනාගතයට මුහුණදෙන්න මොකද්ද samurdhi වෙනස් කරන්නේ මොකක්ද වෙනස් කරන්නේ කුණු දිග වෙනස් කරන්න දිනය උත්තරයක් කියනවද වඩා හොඳින් ඉගෙන ගන්න මීට වඩා කියන එක පිළිබඳ මගේ ලමයන්ට කොහොමද මීට වඩා හොඳ අධ්‍යාපනයක් දෙන්නෙ කියන එක පිළිබඳ කවුරුන් ඔබට උත්තරයක් තියෙන එවුන් පුද්ගලිකව ඉගෙනගන්න කෙනෙක් වෙන්න පුළුවන් ශිල්පියෙක් වෙන්න පුළුවන් නම් මේ ගුරුවරයෙක් විදුහල්පති කෙනෙක් of policy මේක හෙන kotipathi සම්පාදනය ක් වෙන්න පුළුවන් කවුද නෙමේ වීඩියෝ එකට මගේ නම ජ්‍යෙෂ්ඨ කථිකාචාර්යවරයෙක් මම නිතරම හිතන දෙයක් තමයි ගණිතය විද්‍යාව දර්ශණවාදය මින් සංකල්පයන්ගෙන් පැතිර යාම අතිශය වැදගත් අපට රටක් විදෙස් ඔයාට යන්න කියන එක කවුරු දියණියක ශ්‍රී ලංකාවේ අපේ බලය ඇවිල්ලා අපේ මානව සම්පතයි ගොඩනැගිල්ලත් ඒක capitalize කරේ නැත්නම් අපේ රට දියුණුවක් නෑ මේ සංකල්ප මත අපිට පුලුවන්නම් දශම පහක් අපේ සමාජය අඹ ගස් අවුරුදු විස්සක් තිහක් ප්‍රගතිශීලි වෙනසක් අපේ රට දකින්න පුළුවන් වෙයි කියලා ඒ සඳහා තමයි මගේ මේ වීඩියෝ කැප කර මම හිතනවා මේ අධ්‍යාපන ක්‍රමය ගැන අදහස් මේ කතා කරන වීඩියෝ එකක් තිස්සේ වැදගත් වේ ඔබට ඔබේ ජීවිතයේ වගේ ඔබේත් දරුවන්ගේ ජීවිතයේ වගේම මේ කාරණා දැන ගැනීමෙන් මේ පිළිබඳ උනන්දුවක් ඇති කිරීම සඳහා අපි කරන්ඩ දන්නේ සමාජයක් විදිහට හැකිවේවි කියලා මම ක්‍රමය වීඩියෝ ඔබ රසවිඳි කියන්නේ අධ්‍යාපනය ගැන මූලික මට්ටමේ පැහැදිලි අභිලාෂය ගෙනියනවද ඉගෙන ගැනීම අපිට මේ රටක් විදිහට පුද්ගලයෙක් වෙන්න අවශ්‍යතාවය සඳහා සඳනි කොහොමද ලක්ෂපති පැහැදිලි කරලා දෙන්න ඔබට යමක් තියෙනවා අධ්‍යාපන මම දිනුම් වාහනය ඇතුළත කොතනින්ද නවතින්නේ මේ මේසය උඩය ඔබ තියෙනවා දේ ගෙඩියක් ලෙමන් ගෙඩිය පෙනී දුන් ගෙඩියක් ඔය දුව තියෙනවා කියන නම යට මිනිස් උඩ තියෙන නිවලා අඳුර ගන්න කියලා ලමයා හිරේ සහෝදර magadi ගෙඩියක් මේක ලෙමන් ගෙඩියක් මේ ගමන දුරක් එකක් එවන්න වූ නිසා ඔන්න ඔය මට්ටම අධ්‍යාපනය දින ඉතාම මූලික මට්ටම අපි හැමෝම මේකට knowledge ළමයා දන්නවා වෙන් වෙන් වශයෙන් මේ මොනවද කියලා මේ entities මේ දේවල් මොනවද කියලා දන්නවනම් කියනවද යාපනය මූලික මත අඩු අධ්‍යාපනය සැබවින්ම අවශ්‍ය අධ්‍යාපනය තව මට්ටමක් තියෙනවා ළමයින් ගැහුවෙ දෙහි ගෙඩියක් සහ මේ ආවා ලෙමන් ගෙඩියක් අතගා ඔබ හඳුනාගන්න ඔබ තේරුම් යන්නේ කොහොමද වෙනස එකාට කොළ පාට කහ පාට හැබැයි ලෙමන් කියන්නේ එහෙම නෙමේ ලෙවල් දිගටි විශාලයි ඒක සාමාන්‍ය ඉදුනම හොඳටම කහපාටයි වලට වඩා දී ගොඩක් ඇඔුල් වෙමු ඒ උනාට මෙහෙම දෙයක් අධ්‍යාපනය අපි දකින්නේ ඒ දියුණු මට්ටමකට දියුණු තේරුම් ගැනීම පළවෙනි දැනුම දැනුම කියන්නේ මූලිකම වටිනවා නවතම අධ්‍යාපනය අධ්‍යාපනය ඉහළ මට්ටම තමයි knowledge දන්නවා තේරුම් අරගෙන දීමක් මේවා තැන තියෙන වෙනස් කම් සහ මේ විවිධ සංඛධමිම වෙද්දී ගැන කතා කරලා ආර්ථික ගන්න පුළුවන් අහන්න විද්‍යාව වෙන්ඩ පුලුවන් ගණිතය වෙන්න පුළුවන් ගණිත සංකල්ප නැටුම් වෙන්න පුළුවන් සාහිත්‍ය වෙන්නම ඕන දේකට බය කතා වෙන්න පුළුවන් ඉලෙක්ට්‍රොනික උපාංග පිළිබඳව තේරුම් ගැනීමෙන් පැහැදිලි කර දෙන්නම් හරි හරි count of understanding ම niwarana තුන්වෙනි මට්ටම ගමන් කවුද දැන් ළමයා වහනවා හරි දැනුයි මි පැනි දුන්නොත් මොකද පාවිච්චි කරන්න පුළුවන් ළමයා දිව දාන්න එපා අපි පාවිච්චි කරන්නෙ මොකද අඩු නිසා පිළිබඳව ඊට පස්සේ අපි වැඩිදුරටත් නං හැමතිස්සෙම දීමක් විදිහට තබාගන්න කරන තෙරුන් applied වීමේ අවශ්‍යතාවය හැකියාව තියෙන ගැන මේක ගියෙ දෙවියනේ ආකාර යම්කිසි සංකල්පයක් සමේ ස්වභාවය දෙන හැටි ළමයෙක් තේරුම් ගන්න තියෙනවා ඔන්න අධ්‍යාපනය ලද මට එක බුදු නම දැනගැනීමට වඩා ඉහළ මට්ටමේ තේරුම් මෙතනින් නවතින්නේ නැහැ කියලා මෙහෙම හරි ඔයාලට පුළුවන් ද මෙහිදී හිල් එවුනුයි මීපැණි තිබුණු අතර තීරණ වෙනස්කම් විශ්ලේෂණය කළ යුදෙන් තන ළමයා බලනවා හා මොනවද මේ දෙහි ගෙඩියක දින විවිධ දේවල් අනේ picture තියෙනවා ජල මට්ටම් වෙනස් දෙවි ලෙමන් සහ පැණි දොඩම් වල මාට්ටුවෙලා මේක දින ඇඔුල් බව වෙනස් a non ඇඹුල්පේර ගැන කුසින් ටික semata විටමින් සී වගේ දිනලා මැතිරීම් ගන්නකොට වෙනස් වෙන්න පුළුවන් මේක ඇසීම ප්‍රමාණ ඔය වගේ දේවල් වෙනස් වෙන්න පුරමේ ගැනීම විටමින් ramanayeke අත් වෙනස්කමක් මේවා අතර එහෙනම් ලෙමන්වල මෙච්චරයි ඊට පස්සේ dhiwala මෙච්චරයි කියලා මෙන්න මෙහෙමයි මේවා අතර වෙනස්කම් කියලා පොතේ ඝනකම පොත් දෙස භාවය සේරමත් අරගෙන දුන්නොත් අපිට පුංචි එකක් හදලා table එකක් හදලා අහන්න සෞදි මම යනවා පෙරනය දේකට ඒක apply කියන ස්කිල් එකට වඩා ඉහලට මට කියන්න ලංකාවේ ළමයි ඕනතරම් සීගිරිය අදිනවා පොත්වල හා ලෙමන් ගෙඩි කපලා අරගෙන ඇවිල්ලා ආලෙන් අරගෙන එනවා කොට රංචු විලා මම අපි අරගෙන වරෙන් ඇවිල්ල කියනවා හරි ගිහිල්ලා මම කියන මේ බලන්න අපේ නිවාසාන්තර ක්‍රීඩා උළෙල තියෙන ළමයි හොඳක් වෙහෙස වෙනවා මහන්සි වෙනවා ක්‍රීඩා කරන අව්වේ මට ඔයා ඔයාගේ යන ලෙස සැක දිහා බලලා කියන්න මේ ලමයින්ගේ තියෙන පිපාසය නේ වීම සඳහාය ගොල්ලන්ගෙ මේ kandalama අයින් වෙලා යනවා ජල ප්‍රමාණය අයින් වෙලා යනවා ඇඟේ තියෙන මේකට සුදූ මේ විශ්ලේෂණය කරපු එක එක කරුණු දිහා බලලා ඔබ ඒ තෝරා ගැනීම මට පැහැදිලිව කියලා දෙන්න මේක තමයි හොදම එක කියලා පේන්න මේක ඉහළ මට්ටමේ ඊටත් වඩා හරක් analysis කියන එක නම් තොට විශ්ලේෂණයට වඩා ඉහළ මට්ටමේ තියන එක එනවද අන්තිමට ලබාගෙන කියනවා මේ වුනොත් ලෙමන් තියෙනවා දොඩම් තියෙනවා මේ භාණ්ඩ අපේ නිවාසාන්තර දී ඩවුන්ලෝඩ් සඳහා සුදුසු මේ ළමයින්ට ක්‍රීඩා කරන ළමයින්ට සුදුසු බීම ජාතියක් මේකෙ පොටෑසියම් සෝඩියම් සහ විටමින් ජල මට්ටම් සිහිය නම හොඳට කල්පනා කරලා බලලා එකක් හදලා දෙන්න හොඳ වීම ගැලපෙන සහෘද ඔබගෙන් opreation නිර්මාණය පිළිබඳ පස්සේ ළමයා දෙයක් අර සංකල්ප එකට එකට ගන්න කඩල එකට එකට මුහු කරල kauruda nin යමක් ගොඩනඟලා දෙන්න නිර්මාණය කරලා දෙන්න ඕන මම ඔබෙන් හරි ප්‍රශ්න මොකක් ගන්නවද සහෘදයෙක් ලංකාවේ අධ්‍යාපනයේ දෙහි දොඩම් සහ lesson 4 හේතු නම් හඳුනාගැනීම වටේටම යන්ද නැත්තං එය වන්නේ බිම් මට්ටමෙන් ද එවන්න ඒවා තේරුම් ගන්න මට්ටමින්ද නැත්තං ළමයින්ට පුළුවන් දේවල් තියෙන විවිධ වෙනස්කම් දකින්න වෙනස්කම්වලින් සුදුසු දේවල් තෝරලා දෙන්න සහ ඒ වගේම අලුත් නිර්මාණ කරන්න සෞදි විය යුද්ධය ඔබට පමණයි නිර්මාණය මතක තියාගන්න තමයි ඊට පස්සේ understanding ඊට පස්සේ application කියන එක ගත්තොත් අපි සාමාන්‍ය ගෙන මේවා ලොව ඕනෑ ඒ කියන්නේ අඩු මට්ටමේ තියෙනකං නිදි මත අඩු මට්ටමේ තියෙන බුද්ධිමය හැකියාව ඊට වඩා ඉහළ විශ්ලේෂණය ඊට පස්සේ evaluate හොඳම එක තෝරා ගැනීමට සහ යමක් නිර්මාණය කලේ කවුද ඉහළ මට්ටමේ පරිකල්පන හැකියාවන් සැමවිටම අධ්‍යාපනය අවශ්‍යතාවය වෙන්නේ කවුද නිකන් ඔබ හිතන විදියට අර මූලික මට්ටමේ අධ්‍යාපන අවස්ථා කැපවීමද බුද්ධිමය අවශ්යතා සැපිරීම ද රටකට වැදගත් වෙන්නේ අර ඉහල මට්ටමේ අධ්‍යාපන සැමරුම අවශ්‍යතා සැපිරීම chaudhary හොඳම වැඩ මට ඔයා ගන්න එහෙනම් මනුෂ්‍යයෙක් මීට පස්සේ කවදාද වුවද අධ්‍යාපනයෙ කොහෙද මොන ද වෙනස් වෙන්ඩ ඕනේ එහි තියෙන සැබෑ අරමුණ සැප එනවද කියලා මෙන්න මේ framework එක දාල ඔබට උත්තරයක් දෙන්න පුළුවන් සමාජය ගැන කතාව පැත්තකින් තියන්න පුද්ගලයෙක් විදියට ඔබ ඉගෙනගත් දේවල් වලින් ඒ සංකල්ප මගින් තව දින උඹට කොච්චර දෙයක් නිර්මාණය කරගන්න පුලුවන්ද කියන එක මතක තියාගන්න තීරණය වෙනවා ඔබ කොච්චර කල් ලිපියේ duration එකක් ලබාගත හැක ඔබේ දරුවන්ට අවුරුදු සමාජයට යනකොට ඒ දරුවන් තමන් ඉගෙනගත්තු දේ සෑම ප්‍රතිඵලය ලබාගත හැකි වෙන්නේ අර ඉහල මට්ටමේ පරිකල්පන හැකියාව ගොඩනගන්න පුළුවන් වෙන අවස්තාවේදී විදිහට කවුද මේ සැබවින්ම ජනතාවගේ සැබෑ ම වි පෝය අපි හුකාගන්න කවුද අර පහල මට්ටමේ කොමඩු තියෙනවා පහළ මට්ටමේ පරිකල්පනය තුළ නෙමෙයි ඉහළ මට්ටමේ දේවල් අලුත් අලුත් හදන සමාජ අවස්ථාවට අධ්‍යාපනික අවස්ථාවටම සමාජයක් විදිහට මතක තියාගන්න තිස්සේ වැදගත් වෙන මතකද ඉගෙනීම කියලා කියන්නේ ඔය වගේ ප්ලේ ගෙඩිය මේකයි ලෙමන් ගෙඩිය මේකයි දොඩම් ගෙඩිය මේකයි කියලා අඳුරගෙන පොතේ ලියා ගැනීමට ඊට වඩා ඒ සංකල්ප භාවිතා කළ හැකි යමක් නිර්මාණය කිරීම සැබෑ හැකියි අපි නූතන අධ්‍යාපනය කියනවා සැබවින්ම අධ්‍යාපනය ප්‍රතිඵල දායකත්වයක් නිල හොඳට මතක තියාගන්න අති සේවය මල්ලි මම දැන් උඹට කියලා දෙන්න යන්නේ ලෝකේ දැනට අන්දම ගැන තියෙන ප්‍රභලම අධ්‍යාපන ක්‍රමය amayaga ඉගෙන ගත්ත පස්සේ මේක විවිධ ක්‍රමවලට ඔබට ඔබේ දරුවන්ට ඔබ ගුරුවරයෙක් සිසුන්ට මේක අනුගමනය කළ නමුත් මේකට විරුද්ධ using ම problem e නූතන අධ්‍යාපන ක්‍රමය සැබවින්ම අර්ථ දක්වපු මේ පරීක්ෂණයක් ළමයි to section ඒ කියන්නේ මේක කරේ benjamin මේ ගිය විදේශීය වශයෙන් හිටපු ගොඩක් දෙනා පිළිගන්නා පරිදි ඉහළම පුරෝගාමීව අධ්‍යාපනය වූ සහෝදරී ඉතාම ලස්සන පරීක්ෂණයක් කරා ශිෂ්‍යයන් වශයෙන් ඔහු ගත්ත නම් ඓතිහාසික ඉන්න පරිදි තමයි තිහක් ඉන්න පන්ති කාමර ආණ්ඩුව අධ්‍යාපනය අධ්‍යාපනයට නැවත කියද අපෙ ක්ලාස් උඹලගෙ අපි ඉගෙන ගත්ත සාම්ප්‍රදායික අධ්‍යාපනය තමයි ඇවිල්ලා තියෙනවා අද මේ අධි මේ පිටුව මේ පාඩම පෙරලා ගන්න ඊට පස්සේ ගුරුවරයා මොකද කියන්නේ ඒ පාඩම කියෝනව එම නම් බෝඩ් එකේ ලියනවා ගණන් හදලා පෙන්නනවා කතන්දර කියනවා හෙළ පුතේ ගුරුවරයා මොකද කරන්න යන පොතේ තියෙන දේ හරි විෂය කරුණු කෙලින්ම ළමයට උගන් උගන්නලා මොකද කරන්නේ සිංහයාට කියන ඔයා තමයි මේ ගණන් ටික හදාගෙන ඉන්න ඒ වුනත් අම්මේ පැවරුම් ටික කරන්න කියලා කියලා ගුරුවරයායි කාර්යභාරය අවසන් කරල ඊලග පංති වලට යන ලමයි කොහොමද නොවෙන්න පුළුවන් හෙටත් ඔය විදිහටම ගිහිල්ලා අවසානයේදී වාර අවසාන විබාග ලකුණු ගත්තොතින් නැවත කරන ලද පරීක්ෂණ වලදී මෙන්න මේ වගේ කන්නං සාම්ප්‍රදායික අධ්‍යාපන ක්‍රමය පත් නං අපි පතුරු ගන්නේ අපිට පැහැදිලිව පෙනෙන දෙයක් තමයි කවුද මේ අවසාන විභාගය විෂය සඳහා දින විභාගයෙන් ලකුණු අවසාන ලකුණු වල ව්‍යාප්ත වෙන්න මේ වගේ ගිල බෲම් කිව්වා ඒ ගැන ඉතාම අඩු මට්ටමේ ලකුණු ලකුණු තියෙන ළමයි ඉන්නවා ඒ වගේම ඉහල මට්ටමේ ලකුණු දින ලමයින් ඕන නමුත් මේ වී විද්‍යා බලන්ඩ ඉතාමත් පැතිරුණු ව්‍යාප්තියක් සාමාන්‍යයෙන් මධ්‍ය ප්‍රමාණය ලකුණු ඒ ගැන ලකුණු 50 55 වගේ අතර ප්‍රමාණය වැඩිපුර ළමයි ප්‍රමාණයක් ඉන්න ඒ වගේම අඩුම ලකුණු තියෙන ලබාගන්න පාඩුවේ වගේම වැඩීම ලබන තියෙන ළමයි අඩු වෙච්ච මේ වගේ අපගේ නම් බෙල්ල මිෂන් එකක් වගේ සුළු පැහැර බෙල් එකක් ආකාරයෙන් බලන්න අනුමැතිය ඔබත් අත්දැකලා තියෙන පන්ති කාමරවල එහෙම තමයි අඩු ලැබුණු දින ළමයි ඉතින් වත්ද පන්තියේ පළවෙනියා දෙවෙනියා naksha ළමයින්ට ගොඩක් අය මධ්‍ය නම මායා තියෙන ලකුණු ප්‍රමාණය කිව්වා හරි මේක තමයි සාම්ප්‍රදායික අධ්‍යාපනය මම නැවත නැවත පරීක්ෂණ මට පේන්නෙ මේ සාම්ප්‍රදායික අධ්‍යාපනය පාඩම කියලා දෙනවා පැවරුම් දෙනවා නමුත් පැවරුම් මට ලොකු feedback යකෙක් ගුරුවරයා මේක මෙහෙම කියලා කියන්නේ නෑ ළමයි ගණන් හදනවා එක තමුන්ට හැකියා දින ප්‍රමාණයට මේක නෑ ඒ වගේම අවසාන වාර විභාගය ගත්තොත් එම මේ වගේ දේවල් ලකුණු පෙන්නන්න ඉගෙන ගන්න පුළුවන් ළමයි ඉගෙන ගන්න බැහැ ළමයා බැලුවා නෑ ඔහොම නෙමේ මේ විදියට උකුනු ඒ කියන්නෙ ටෙලි නාට්‍ය ගුරුවරයා ඉගැන්වීමේ ක්‍රමය අරගෙන යන්නේ ළමයි සෑහෙන ප්‍රමාණයක් ඒ සුදුසු ප්‍රවීණත්වයට පත්වූ අය ලස්සනට ළමයි තිහයි එකම ගුරුවරයා හැබැයි ගුරුවරයා එයා කරන උගන්වන ක්‍රමය පොඩ්ඩක් වෙනස් කරන එක තමයි ඔන්න අද ඇවිල්ල පාඩම කියලා දෙන පාඩම කියලා දෙනවා පාඩම මේ ආකාරයට කලින් විදියටම තමයි හැබැයි ඒ පන්තිය අවසානයේදී ළමයින්ට දෙන පුංචි පැවරුමක් පිලට ඒ එද්දි වීදුරු පුංචි පැවරුමක් දී ළමයින්ට ගිහින් මුහුදේ ගණන් හදන්ඩ පොළියක් දෙනව ළමයින්ගෙ ලකුණු එකතු කරගන්නවා බලන්න බැලුවම ග්‍රේරු යනවා වෙන්නේ මේ කාරණා ලමයන්ට pay පැහැදිලි වෙලා නෑ මේක හොඳට පැහැදිලි තියෙනවා එහෙනම් මම තව ටිකක් මේ මට්ටමේ මේ මේ මේ සංකල්ප වෙච්චර තේරුම් ගත්ත නැති සංකල්පය තව ටිකක් වෙලාව යොදවන තව ටිකක් තේරුම් කරන්න ඔය විදියට නවදෙනෙක් මාස්ටර් ලයින් ඒ ගැන ගුරුවරයා ගමන් කරන්නේ විෂය හරහා ගමන් කරන්නේ ළමයාව නැවත නැවත පරීක්ෂාකර කරන අන්තිම හරි බාගෙන විතරක් නෙවෙයි බලල ගුරුවරයා තේරුම් ගන්න මම කියපු දේවල් කොච්චර ළමයි තේරුම් අරගෙන තියෙනවද නිවැරදිව පාවිච්චි කරන්න පුළුවන්ද කියලා තේරුම් අරගෙන ඒ දුර්වලතා මගහරිමින් free පද්ධතියේ ම නැතත් නැවත නැවත එය සුදුසු වෙනස්කම් කරමින් ඒ ගැන කියන අර කලින්ම ළමයි ටිකක් තමයි ඒ ළමයගෙ විෂය තියන ප්‍රවීණත්වය සෞදි මට පුළුවන් දකුණු අත පැත්තට කරන්න ඒ ගැන සාමාන්‍ය දීගෙක වගේ තිබුණු ළමයෙක්ට එකකට push කරන්න පුළුවන් ඒ වගේම අර කලින් ෆේල් වෙච්ච ළමයින් වන රැගෙන ලකුණු තිහකට වඩා අඩු ළමයි ඉන්න ඕනේ ඒ තිහකට වඩා ළමයි සහෝදර අඩු වශයෙන් බී එකක මට්ටමේ මටත් පුස් කරන්න පුළුවන් සාම්ප්‍රදායික අධ්‍යාපන මෝඩයෝ කියලා හැරිලා යන ළමයි තරුදිනි මෝඩයො වුණේ වෙන්ඩ මෙන් ඔවුන් කියන ආකාරයට එහෙනම් අධ්‍යාපන ක්‍රමයේදී වන වැරැද්දක් මාස්ටර් එළනේ කියන්නේ විෂය ගෙනියන්නේ ළමයින් යම්කිසි ප්‍රමාණයකට පැමිණි පසුවද my ලඟ විශේෂයක් ඊළඟ පාර මට යන්නේ ඒ පාඩම අවශ්‍ය ප්‍රවීනත්වයට පැමිණ වුණාම තමයි ඊළඟ පාර මැද්දේ නං සැබවින්ම අරන් දුර්වල ළමයාවත් අරගෙනම ගමන යන්න පුළුවන් වෙන මට්ටමට වඩා ඉහළ මට්ටමේ හෙම විජම් ඉල්ලුම් අවසානයේදී කිව්වේ හෝ කිව්වා වූ වෙනස් කරා දැන් පන්ති කාවද ක්‍රමය වෙනස් කරන සෑම ළමයි තුන් දෙනෙක්ටම ගුරුවරයෙක් ගහන දෙනවා තුන්දෙනාට ගුරුවරයෙක් ගහන දෙනවා බීලා අර ළමයි වෙනස් කරන්න දන්සල මැතිව් ලන්නට සමත් ළමයි තුන්දෙනෙක් දෙනවා ලබුනෝරැව ළමයි දෙන්න කියනවා ඔයාලා පාඩම කරන්නා සේ එකම පාඩම තමයි ඒ පාඩම ඉදිරිපත් කරන්න හැබැයි ඉදිරිපත් කරන කොට සෑම විටම ඇගයීම් කරන්නං ළමයි සුන්දරයි අගයන්නේ ළමයි තුන්දෙනාගේ මට්ටම නිතරම බැරිනම් එකෙන් අභිෂේකාගේ ග්‍රහණය කරගන්න විෂයක් ඒ යම්කිසි කොටසක් අමාරු නම් එයාලට personalized පිහාටු පුද්ගලිකව වෙච්ච අවස්ථා උදව් කිරීම එළම ඉදිරියට හිමියන්ට එක මාස්ටර් ලයින් වලට වඩා වෙනස් දැන් තේරුනා නේද මේක පෝර සරණයි හර්නියා මෙහෙම uterine ගෙවා ඒ ගැන ඒ ළමයාට අවශ්‍ය පුද්ගලික වශයෙන් budis ලබාදෙන්න පාඩම් කරගෙන යනවා මං ඒ ගැන එකට එක වශයෙන් ඒ ගැන අවශ්‍යතාවයෙන් ළමයාගේ ප්‍රවීණත්වය ලැබෙන කං මේ ළමයාට පුද්ගලිකව උදව් ලබා දෙමින් ඉදිරියට මේකෙන් එහෙම මඬුවක් nema නම් අන්තුවා tutoring ගෙන් අවස්ථාව කවුරු දෙනෙක් one-to-one tutoring මට්ටමට අපේ අධ්‍යාපන ක්‍රමය කරත් දෙන්නේද යන්න පුළුවන් නම් මොකක්ද වෙන්නේ බෙන්ජමින් බ්ලූම් කිව්වත් using a canon බිහි අකුර තියන ළමයින් නෝනා බී අකුරෙන් ළමයි නෝනා මේ ගොඩේ ප්ලස් එකක් දක්වා ගෙනියන්න ඒ වගේම යන්තමින් වශයෙන් නොගෙන ළමයා එකඟ වලකින්න තමයි ඉන්නේ එක ඉඳලමයි එකක් දක්වා වීරයන් 75 දක්වා අරගෙන යන්න පුළුවන් ළමයි ප්‍රමාණය සෞදි 0 දක්වා අරගෙන යන්න පුළුවන් ස්ටර් එය ඉහළ mas melon in වලට වඩා ඔබ්බට ගිය වන් රුවන් tutoring තොර බෙන්ජමින් බ්ලූම් මොකද්ද කවුද නෙමේ කියන්නේ මේ ලෝකෙ අධ්‍යාපන ක්‍රමයේ ඇතුලෙන් බිහි වෙනවා නම් වෙන්න තියෙන හේතුව තමයි සැබැවින්ම ළමයාගෙන් ක්‍රමයක් තියෙනවා මේ දුර්වලයන් ව නැති කළ සම්පූර්ණයෙන්ම a master රියක විෂය සඳහා දිය නෑ ඒ ප්‍රවීණත්වය ළමයට ලබාගන්න හැබැයි ඒ සඳහා එක් එක් ළමයාගේ ප්‍රවීණත්වය ලබා ගැනීම සඳහා වස්තු හැකියාවන් එකිනෙකට වෙනස් වෙසක් එකෙක් ළමයාට individual අයිසෙජ් එහෙම නැත්තං අපි මේ ගැන කියනවා ශිෂ්‍ය කේන්ද්‍රීය අධ්‍යාපනය කියලා ඒ වචනය සංඛාර ඇති නමුත් මේකෙ තේරුම ගොඩක් වෙලාට භාවිතා වෙනවා ඉතාම අඩු විශාරද බලන්න මේ ශිෂ්‍ය කේන්ද්‍රීය අධ්‍යාපනය ශිෂ්‍යයාගේ වේගයට තමයි මේ දැනුම අපි ඔවුන්ට ලබා දෙන්නේ අර කතාවක් තියෙන්නේ නම් මම ඒක දැකලා තියෙනවා ගොඩක් වෙලාවට පංතිකාමරය දුර්වලම නම්බර් එක ලියාගන්න කලින් ගුරුවරයා පන්ති ම ගල්ලෑල්ල පුවරු ක කවුරුත් ඉන්න භෞතික වශයෙන් නෙමෙයි ඊට වඩාත් සංකල්පීය වශයෙන් සත්‍ය නිර්වාණය ද එ්ක යන්නේ මාස්ටර් දේ ගැන ඒ ළමයා එකෙක් ළමයා ගැන දුර්වලම නෑ මෙයා විෂය අල්ලගන්න කලින් ගුරුවරයා ඊළඟ විෂයට ගියා අනින්න ඒක නිසයි තව ළමයෙක් සමාජ එහෙම නැත්නම් සහෘද නිදන්න මේ අධ්‍යාපන ක්‍රමය මාස්ටර් ඉන්න කෙනෙක් ප්‍රවීණත්වය මත පදනම් විද්‍යා අධ්‍යාපනය ක්‍රමයේ හෝ එහෙම නැත්තං කෝල් ශ්‍රී කේන්ද්‍රීය අධ්‍යාපනයට අපි ගෙනිච්චා නම් සම්ප්‍රදායක අවසාන සමයේදී ඉහළම ලකුණු ගත්තු 20% ඉන්නව නේද හළම ලකුණු ගත් සියලු විසින් වන සත්තු ප්‍රමාණයේම marsilan in වල සහෝදර 70 ශිෂ්‍ය කේන්ද්‍රීය අධ්‍යාපනය ක්‍රමය වන් රුවන් uterine carrers nice education වලින් සෞදි සීයට අනූවක් අර කලින් සාම්ප්‍රදායික අධ්‍යාපන ක්‍රමයේ සීයට බිෂූ ලකුණු ප්‍රමාණය ගන්න වෙලා සහෝදරී ඉතාම වැදගත් කාරණා මම ඒ නයිල් ලඟට කියල දෙන්නම් කොහොමද මේ ශිෂ්‍ය කේන්ද්‍රිය එම නැත්තං උඹ දැන් උඹ ඉගෙන ගන්න ගෙන ඔබ කේන්ද්‍රගත වෙච්ච් අධ්‍යාපන ක්‍රමයක් ඔබ ලබා ගන්නේ කොහොමද ඒකට දෙන ප්‍රධාන මූලධර්ම පැහැදිලි කළ දෙන්නම් ඒ සපුරා ගැනීම සඳහා දින මම ඊළඟට පැහැදිලි කරල දෙන්න මේ ශිෂ්‍ය කේන්ද්‍රිය අධ්‍යාපනයේ මූලධර්ම දැන් ඔබේ දරුවාට එම නැත්නම් උඹට ඔබ ගුරුවරයෙක් නම් උඹේ සිසුන්ට උඹ කොහොමද මේ ඉහළ මට්ටමේ අධ්‍යාපනය ලබාදෙන්නේ මේකේ මූලධර්ම මොනවාද පළවෙනි ඉතාම වැදගත් මූලධර්මය තමයි සහෘදයින් continuous playback නිත්තම මේ ළමයාගේ එවන තනේ යම්කිසි විෂයයක් පිළිබඳ ප්‍රවීණත්වය නිතරම මෙම බලන්ඩ අවසානය වහිනවානේ නිතර නිතරම හිතන්නේ ලොකු පාඩමක් ගැන කියලා දෙන්න ගැන කියලා දෙන්න ත්‍රිකෝණමිතිය කියන පාඩම කියලා දෙනවා කියලා හිතන්න තරුදිනි මුලින්ම එය මුළු ත්‍රිකෝණමිතිය පාඩම අවසාන කරන වාර විභාගයට නෙමේ ලකුණු ප්‍රශ්න අහන්නේ යාමේදී පහරදෙනවා පාඩම හදලා ඒ එක් එක් අවස්ථාවල තියෙන ප්‍රවීණත්වය සෞදි milo තේරුනෙත් අන්න ඒක තමයි කන්නේ ප්‍රවීණත්වය මණ්ඩල ඒ තියෙන යම්කිසි අභියෝගයක් තියෙනවා නං ඒ ඒ ග්‍රහණය කර ගන්ඩ විෂය ඒ අභියෝගයට සුදුසුම ඒ අභියෝගය මඟහරවා ගැනීම සුදුසු උපදෙස් ලබා දෙන පොත් විශේෂම continuous assessment නිතරම ළමයා ඇඟවීම ඇගයීමට අනුරූපව යම්කිසි පීඩකයා කෙමනට යම්කිසි මෙය උපදෙස් ලබාදීමත් තිස්සේ වැදගත්කම vinta අංක එක අංක දෙක තමයි how අවස්ථාව ලබා දෙන්න ඕනේ ඒ තියෙන විශේෂ මුන්ට පුළුවන් වේගෙන් හැරීම සඳහා ගෑනි දන්න සාම්ප්‍රදායික අධ්‍යාපනය ගුරුවරයෙක් පාඩමක් ගැලවෙන්නම කියලා දෙනවා a lecture එක ඇතුලෙදි යනවිට ළමයට මැටි කරන්න බැරි ළමයට විෂයයක් මගහැර දානය රැගෙන පාඩම ඇතුලේ දේශනයක් දින දින කාරයි වෙනුවට සහෘදයනි හැකියාව තියෙනවා නම් බලන්න දේශණය එතකොට ළමයට අර යම් කිසි කාරණයක් තේරුනේ නැත්නම් නැවත නැවත ගිහිල්ලා ඒක නැවත නැවත පරිශීලනය කළා යැයි ප්‍රවීණ බෙදන්න පුළුවන් වෙනවා නේද අවස්ථාව වගේ නෙමේ අර තියන විශේෂම ඒ ඒ ළමයා ගැටෙන ප්‍රමාණය වැඩි වෙනවා english font එක ගැන ගැටීම ප්‍රමාණය වැඩි කරන්න පුළුවන් මේක දෙවැනි මූලධර්මය අතිශය වැදගත් මේ අපි ගැනම individual අයිස් එම නැත්තං මේ student සෙන්ටර් එක්ක මේ අධ්‍යාපන කරමින් තුන්වෙනි ඉතා වැදගත් මූලධර්මයක් තමයි අපි මේ විශේෂයෙන්ම මේ ළමයින්ට උගන්වන කොට මේ ක්‍රමයේ ඇතුළත ළමයින්ට පුංචි මේ සෑම විටම ක්‍රියාකාරකම් ලබාදීම වැදගත් activities එම ප්‍රවෘත්තිය ලබාදීම මතකනේ අපි කතා කරන්න අධ්‍යාපනය ඉහල මට්ටම් තියෙනවා කියලා නිකං ලෙක්චර් එකක් adare පාඩමක් කියල දෙනකොට ගොඩක් වෙලාවට අපිට ලබා දෙන්න පුළුවන් අර මූලික ආකාරය අධ්‍යාපනය තමයි ඒ ගැන knowledge therstan නිර්වචන අප්‍රායෝගික මත ලබා දෙන්න හැකියාව තියෙන ක්‍රියාකාරකම් ළමයින්ට දෙනවා දෙකක් දීලා මේවා අනුපාත මල්ලි මිශ්‍රකර බීම හදල බලන්න මේකෙන් හොඳම දේ හෝ හොඳම මී මේක හොයාගන්ඩ කිව්වා දින උඹලට පේනවද ළමයා ඊට වඩා විශ්ලේෂණ කිරීමේ හැකියාව යන්ඩ ළමයා විසින් ම ය කියන්නේ ඒ වගේ මේක රසවත් ක්‍රියාකාරකමක් ළමයාගේ ඒ විෂය සමග ගැටීමේදී naha kiya වැඩි කරනවා අර දුර්වලම නව යෞවන පන්තිය කවුද ඒ වෙලාවේ විශේෂම ගැටෙන ප්‍රමාණය වැඩි තේරුණාද dhinam අද විද්‍යා ගන්න ඔය අපි කියනවා activities education එවන තැන් project based education කියලා කියන්නේ ඔය අධ්‍යාපන ක්‍රමේ බලන් නැද්ද මේ දුර්වල නම් යම්කිසි විෂයක් සහ දුර්වල නම් ඒ ගුරුවරයා සමග කතාබහ කරන සහෝදර මිය වගේ ක්‍රියාකාරකම් වලට ඒ ළමයා යොමු කිරීම වටිනවා ඔබටත් එහෙම නිතරම කියන දෙයක් තමයි විශේෂ කුඩා ළමයයි නිතරම ගෙවත්තේ දෙමාපියන් සමග පුංචි පුංචි activities of science activities වලට දෙන්න පුංචි කැමරා එකක් දීලම ෆෝන් එකක් දීලා මේ උඩවත්ත ඇතුලේ තියෙන විවිධ ලයිකන වර්ග හොයන්න ළමයා දීමට කරගෙන ෆොටෝ අර ගහලා ඒ ෆොටෝ අන්තිමට ඒ ෆොටෝ වල තියෙන දේවල් කෘමීන්ගේ විවිධත්වයන් කොහෙද මේගොල්ලෝ දැක්කේ ඒගොල්ලො මොනවගේදැයි ගුණ ජීවන චක්‍රය කියලා අපිට පැහැදිලි කරන්න අවස්ථාවක් ලබා දෙන්න දෙමව්පියන්ට අපිට අධ්‍යාපනය ගොඩනඟන්න පුළුවන් හැකියාව මෙතෙක් ඒ වගේම සමුදය මී අධ්‍යාපන ක්‍රමයේත් අවසාන මූලධන වැදගත් එකම කියලා දෙන්න තව කීපයක් දීලා නම් මූලික දේවල් apiyana යුද්ධයෙන් ට නෙමේ ෆේස් ටු ෆේස් අපි අර මුහුණට මුහුණ කරන සාමාන්‍ය වලින් එකක් හරි කරන සාමාන්‍ය සාම්ප්‍රදායික අධ්‍යාපන ක්‍රමයට උඹට ගිහිල්ලා blender ඒ ගැන හිතන්නෙ ළමයාට යමක් දෙනවා හොයාගෙන එන්න මේ ඉන්ටනෙට් එක ගවේෂණය කළ අන්තර්ජාලය කරන මේ විෂය කියලා දෙන්න මූලික සංකල්පයක් එක දෙන්න පුළුවන් ළමයාට දෙනුව හොයාගෙන ඉන්න එතකොට හිටියේ ඒ කිය අපිට පහණ පුළුවන් එක understand මට්ටමේ අහන්න පුළුවන් ඊට පස්සේ යන්නේ lies මට්ටමේ අහන්න පුළුවන් සමාන රට ක් විය යුතු මට්ටමේ වුන මේ වගේ දෙයක් ඔයා අලුතෙන් හිතලා මේ ප්‍රශ්න විසදීම සඳහා හොඳ එකක් solution එකක් ඔයා internet එකේ search කරලා රජුන් එන්ඩ කිව්වොත් අපි කරන්න මට්ටම සාමාන්‍ය පංති කාමරයක ගොඩනගන්න බැරි දැනුමක් ඒ විකුණ ගන්නවා ලේනෙක් සාමාන්‍ය සාම්ප්‍රදායික අධ්‍යාපනය ඔබ්බට ගිහිල්ලා ළමයාට අන්තර්ජාලය සහයෝගය අධ්‍යක්ෂණය ඇසුරු කරගෙන ක්‍රියාකාරකම් වලට job පැමිණිය සෞදි මතක තියාගන්න අතිශය වැදගත් ්වා නේද එහෙනම් අහිමිවීම් මේක අපේ කද වීඩියෝ එකට එනවද මීට කතා කරන්න බෑ අධ්‍යාපනය ගැන මම සැබෑ අධ්‍යාත්මික වල ගොඩනඟන්නේ කොහොමද ඒ වගේම එහි ප්‍රධාන මූලධර්මයන් කිහිපයක් ඔබට අඳුනගන්න ඔබේ ළමයාට එම දරුවාට ලබාදීම බොහෝම ස්තුතිවන්ත වෙනවා මේ සඳහා සහභාගි වීම සම්බන්ධයෙන් මේ වගේ වැදගත් කාරණාවක් යම් යම් සමූහ එතකල් බෝම ස්තූතියි වැඩසටහන් සමඟ සම්බන්ධ වුණා
"""

print( summarize_text(text6))

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [ ]:
download_file('/kaggle/working/results/checkpoint-966', 'out')

In [ ]:
import os

# Specify the file path to delete
file_path = '/kaggle/working/out.zip'

# Check if the file exists and delete it
if os.path.isfile(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")